In [11]:
import os
import requests
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from zipfile import ZipFile
from google.cloud import storage
from google.oauth2 import service_account

load_dotenv()

ROOT = Path.cwd().parent
DATA_DIR = ROOT / "data"
DATA_DIR.mkdir(exist_ok=True)

PROJECT_ID = os.getenv("PROJECT_ID")
SA_PATH = ROOT / os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
BUCKET_NAME = os.getenv("BUCKET_NAME")


In [2]:
# Authentification Google Cloud
creds = service_account.Credentials.from_service_account_file(SA_PATH)
storage_client = storage.Client(project=PROJECT_ID, credentials=creds)

# Ingestion des données Historiques de validation

In [4]:
URL = "https://data.iledefrance-mobilites.fr/api/explore/v2.1/catalog/datasets/histo-validations-reseau-ferre/records"

response = requests.get(URL, timeout=60)
response.raise_for_status()

records = response.json().get("results", [])
print(f"Nombre de fichiers trouvés: {len(records)}")

df_metadata = pd.DataFrame([
    {
        'annee': int(r['annee']),
        'filename': r['reseau_ferre']['filename'],
        'url': r['reseau_ferre']['url']
    }
    for r in records
])

print("\nFichiers disponibles:")
display(df_metadata)

Nombre de fichiers trouvés: 10

Fichiers disponibles:


,annee,filename,url
0,2017,data-rf-2017.zip,https://data.iledefrance-mobilites.fr/api/expl...
1,2019,data-rf-2019.zip,https://data.iledefrance-mobilites.fr/api/expl...
2,2020,data-rf-2020.zip,https://data.iledefrance-mobilites.fr/api/expl...
3,2021,data-rf-2021.zip,https://data.iledefrance-mobilites.fr/api/expl...
4,2016,data-rf-2016.zip,https://data.iledefrance-mobilites.fr/api/expl...
5,2018,data-rf-2018.zip,https://data.iledefrance-mobilites.fr/api/expl...
6,2022,data-rf-2022.zip,https://data.iledefrance-mobilites.fr/api/expl...
7,2023,data-rf-2023.zip,https://data.iledefrance-mobilites.fr/api/expl...
8,2015,data-rf-2015.zip,https://data.iledefrance-mobilites.fr/api/expl...
9,2024,data-rf-2024.zip,https://data.iledefrance-mobilites.fr/api/expl...


In [ ]:

BASE_DIR = DATA_DIR / "histo-validations-reseau-ferre"
BASE_DIR.mkdir(parents=True, exist_ok=True)

for row in df_metadata.itertuples():
    year_dir = BASE_DIR / str(row.annee)
    year_dir.mkdir(exist_ok=True)
    zip_path = year_dir / row.filename

    print(f"Téléchargement {row.annee} ({row.filename})...")
    try:
        response = requests.get(row.url, timeout=300)
        response.raise_for_status()
        
        zip_path.write_bytes(response.content)
        print(f"Téléchargé")
        
        with ZipFile(zip_path) as zf:
            zf.extractall(year_dir)
        print(f"Extrait dans {year_dir}")
        
        zip_path.unlink()
        
    except Exception as e:
        print(f"Erreur {row.annee}: {e}")

print(f"\nTerminé: {BASE_DIR}")


Téléchargement 2017 (data-rf-2017.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2017
Téléchargement 2019 (data-rf-2019.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2019
Téléchargement 2020 (data-rf-2020.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2020
Téléchargement 2021 (data-rf-2021.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2021
Téléchargement 2016 (data-rf-2016.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-validations-reseau-ferre/2016
Téléchargement 2018 (data-rf-2018.zip)...
Téléchargé
Extrait dans /Users/admin/Desktop/projects/m2-univ-reims-sep-cs-etl-sncf-gcp/data/histo-vali